<a href="https://colab.research.google.com/github/TomoharuKurosu/TomoharuKurosu/blob/main/growth_comp_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt90.loc[pwt90['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share
data['A'] = data['y_pc'] - data['k_pc']*data['a']

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['y'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['k'] = (grouped_data['k_pc'].diff() * 100)  # Growth rate of GDP per capita
data['AA'] = (grouped_data['A'].diff() * 100)  # Growth rate of GDP per capita


# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'y': 'mean',
                                           'labsh': 'mean',
                                           'AA': 'mean',
                                           'k': 'mean',
                                       'a': "mean"}
                                       )

# Calculate additional summary statistics
summary['Growth Rate'] = summary['y']
summary['Capital Share'] = summary['a']
summary['TFP Share'] = summary['labsh']
summary['TFP Growth'] = summary['AA']
summary['Capital Deepening'] = summary['k']

# Print output
print(summary)

                    y     labsh        AA         k         a  Growth Rate  \
countrycode                                                                  
CAN          0.744709  0.615643 -0.786829  1.399592  0.384357     0.744709   
DEU          0.689737  0.625118 -2.591309  1.099346  0.374882     0.689737   
FRA          0.818749  0.621001  0.949124  1.400636  0.378999     0.818749   
GBR          1.104174  0.620237  1.934368  0.685596  0.379763     1.104174   
ITA          0.043478  0.529206 -0.567753  1.322566  0.470794     0.043478   
JPN          0.909459  0.620112 -4.334671  2.452127  0.379888     0.909459   
USA          1.589208  0.616466  0.343034  1.571761  0.383534     1.589208   

             Capital Share  TFP Share  TFP Growth  Capital Deepening  
countrycode                                                           
CAN               0.384357   0.615643   -0.786829           1.399592  
DEU               0.374882   0.625118   -2.591309           1.099346  
FRA          